In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

## Load the dataset.
You'll need to customize the file location, of course.

In [ ]:
colin_icbm = np.fromfile('/home/welling/git/CMU-MS-DAS-Vis-S22/data/colin27_icbm_181_217_181.bytes', dtype=np.uint8)

colin_icbm = np.reshape(colin_icbm, (181, 217, 181), order='F')

## Mark the right side of the data

Front and back, top and bottom are obvious, but left and right can be confusing.  Let's make a visible mark.  By the "radiological convention", low first index (low X coordinate) corresponds to the patient's right side.

In [ ]:
colin_icbm_marked = colin_icbm.copy()
colin_icbm_marked[5,:,:] = 200  # insert a plane of high values at low first index

## Draw a simple contour plot

In [ ]:
x = np.arange(0.0, colin_icbm_marked.shape[0], 1)
y = np.arange(0.0, colin_icbm_marked.shape[1], 1)
X, Y = np.meshgrid(x, y, indexing='ij')
print(colin_icbm_marked.shape)
print(X.shape)
print(Y.shape)

In [ ]:
fig, ax = plt.subplots()
countours = ax.contour(X, Y, colin_icbm_marked[:,:,50])


See the marker plane above, at low X?

## Create a function to draw the slice.
Now we start building some code complexity.  The first thing we want to fix is the aspect ratio of the figure.

In [ ]:
def draw_slice(slice_num):
    fig, ax = plt.subplots()
    contours = ax.contour(X, Y, colin_icbm_marked[:,:,slice_num])
    ax.set_title(f'slice {slice_num}')
    # set the aspect ratio
    ratio = colin_icbm_marked.shape[1]/colin_icbm_marked.shape[0]
    x_left, x_right = ax.get_xlim()
    y_low, y_high = ax.get_ylim()
    ax.set_aspect(abs((x_right - x_left)/(y_low - y_high))*ratio)
                  
    
    

In [ ]:
draw_slice(50)


## Create a factory function.
More complexity.  We want to be able to redraw a slice in response to requests from widgets, without
recreating the figure or axes.  Axis.clear() removes the old contents.  (There is some discussion about whether it is sufficient to free the memory, but it does enough for this demo).

We are using a 'closure' to associate the Figure and Axis with the redraw functions.

Read about the use of Traitlets in ipywidgets to better understand the code in redraw_observer_func().

In [ ]:
def create_draw_slice(vol, axis):
    assert axis in [0, 1, 2]
    idx0, idx1 = [(1, 2), (2, 0), (0, 1)][axis]
    axis_name = ["X", "Y", "Z"][axis]
    X, Y = np.meshgrid(np.arange(0.0, vol.shape[idx0], 1.0),
                       np.arange(0.0, vol.shape[idx1], 1.0),
                       indexing='ij')
    fig, ax = plt.subplots()
    ratio = vol.shape[idx1]/vol.shape[idx0]
    x_left, x_right = ax.get_xlim()
    y_low, y_high = ax.get_ylim()
    ax.set_aspect(abs((x_right - x_left)/(y_low - y_high))*ratio)
    def redraw_func(slice_num):
        ax.clear()
        ax.set_title(f"{axis_name} slice {slice_num}")
        if axis == 0:
            contours = ax.contour(X, Y, vol[slice_num,:,:])
        elif axis == 1:
            contours = ax.contour(X, Y, vol[:,slice_num,:].transpose())
        else:
            contours = ax.contour(X, Y, vol[:,:,slice_num])
    def redraw_observer_func(slice_num_bunch):
        """
        The paremeter is a 'traitlets.utils.bunch.Bunch'.  This is
        a strange feature of the way ipywidgets are implemented.
        """
        redraw_func(slice_num_bunch.new)
    return fig, ax, redraw_func, redraw_observer_func


In this next block we are seeing some distortion of the aspect ratio, despite the care we've taken.  I'm not sure why.

In [ ]:
z_fig, z_ax, z_redraw_func, redraw_observer_func = create_draw_slice(colin_icbm_marked, 2)
for i in range(10):
    z_redraw_func(40+10*i)

## Demonstration of ipywidgets.interact()

In [ ]:
z_fig, z_ax, z_redraw_func, redraw_observer_func = create_draw_slice(colin_icbm_marked, 2)
widgets.interact(z_redraw_func,
                 slice_num=widgets.IntSlider(value=50, min=0, max=colin_icbm_marked.shape[2]-1))


## A demonstration of using widget.observe

Note that the first view doesn't get drawn until we move the slider.

In [ ]:
z_fig, z_ax, z_redraw_func, z_redraw_observer_func = create_draw_slice(colin_icbm_marked, 2)
z_slice_slider = widgets.IntSlider(value=50, min=0, max=colin_icbm_marked.shape[2]-1)
z_slice_slider.observe(z_redraw_observer_func, "value")
z_slice_slider  # return the slider to cause it to be displayed

## Now let's construct a unified display tool.
It doesn't fit well in the notebook's display area, but you can see that it's working and that it could be rearranged.

In [ ]:
output_x = widgets.Output()
with output_x:
    x_fig, x_ax, x_redraw_func, x_redraw_observer_func = create_draw_slice(colin_icbm_marked, 0)
output_y = widgets.Output()
with output_y:
    y_fig, x_ax, y_redraw_func, y_redraw_observer_func = create_draw_slice(colin_icbm_marked, 1)
output_z = widgets.Output()
with output_z:
    z_fig, x_ax, z_redraw_func, z_redraw_observer_func = create_draw_slice(colin_icbm_marked, 2)


In [ ]:
x_slice_slider = widgets.IntSlider(value=50, min=0, max=colin_icbm_marked.shape[0]-1)
x_slice_slider.observe(x_redraw_observer_func, "value")

y_slice_slider = widgets.IntSlider(value=50, min=0, max=colin_icbm_marked.shape[1]-1)
y_slice_slider.observe(y_redraw_observer_func, "value")

z_slice_slider = widgets.IntSlider(value=50, min=0, max=colin_icbm_marked.shape[2]-1)
z_slice_slider.observe(z_redraw_observer_func, "value")


In [ ]:
widgets.HBox([widgets.VBox([x_slice_slider, y_slice_slider, z_slice_slider]),
             output_x, output_y, output_z])